# Probabilistic sensitivity analysis: uniform distributed parameters
In probabilistic sensitivity analyses distributions of baseline parameters are resampled multiple times from a distribution. Subsequently, the Markov model is implemented for the defined simulation period and cumulative outcomes for the cohort are computed (QALYs: Quality-Adjusted Life Years, costs in €, NMB: Net Monetary Benefit in €). This is often referred to as the term Markov Chain Monte Carlo simulations. The computations below implement draws of model parameters from a uniform distribution between minumum and maximum values of the model parameters. The data used refers to the study (appendix D includes all parameters used): <br>

"Cost and health effects of case management compared to outpatient clinic follow-up in a Dutch heart failure cohort" <br> by H. van Voorst and A.E.R. Arnold.<br>
DOI: 10.1002/ehf2.12692 <br>

This notebook is the second in an series of three:
1. Baseline simulations and one-way deterministic sensitivity analyses.
2. Probabilistic sensitivity analysis: uniform distributed parameters
3. Probabilistic sensitivity analysis: most probable distributed parameters

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
np.random.seed(24)
import math
import time

## Probability-time adjustment functions
Since the baseline input values of each baseline parameter was not estimated over the same time span computations were required. Furhtermore, based on a probability of an event in a control arm the probability of an event in the intervention arm was computed with the Relative Risk (RR). Functions below assume constant distribution of probabilities through time.

In [2]:
# Computing with relative risks
def RR_intervention(p_control, RR, rr_months, pc_months):
    """
    Computes the probability of an event 
    in the intervention arm based on 
    the Relative Risk (RR) and probability
    in the control arm (p_control).
    
    p_control: Probability of event in control arm
    RR: Relative Risk of event in intervention 
    arm relative to control arm
    rr_months: months used to compute RR
    pc_months: months over which p_control is measured
    """
    
    if rr_months==pc_months:
        p_intervention = p_control*RR
    else:
        # first convert the control probability to the 
        # same follow up time of the RR probabilities
        pc_adj = 1-(1-p_control)**(rr_months/pc_months)
        # compute the intervention probability event free
        p_int_eventfree = 1-pc_adj*RR # probability of no event in intervention group
        # go back to the followup time of the control probability
        p_intervention = 1-p_int_eventfree**(pc_months/rr_months)
    return p_intervention


## Draw all paramters
The 

In [16]:
#Function that draws from uniform distributions the parameters
def draw_unif_probabilities():
    #control group
    #RR_readmission,RR_mortality=0.75,0.77
    RR_readmin, RR_readmax, RR_mortmin, RR_mortmax = 0.53,0.78,0.68,0.90
    RR_readmission =  np.random.uniform(low = RR_readmin, high = RR_readmax)
    RR_mortality =  np.random.uniform(low = RR_mortmin, high = RR_mortmax)

    #B = N12 -> N34 (NYHA decay from NYHA 1/2 to 
    #NYHA 3/4; net effect assumed zero)
    bc_draw = 0
    bi_draw = 0

    #C = N12 -> H (Hospital readmission from NYHA 1/2)
    cmin = 0.0044
    cmax = 0.0486
    # convert min-max values with RR's
    cimin = RR_intervention(cmin, RR_readmission, 12, 1)
    cimax = RR_intervention(cmax, RR_readmission, 12, 1)
    cc_draw = np.random.uniform(low = cmin, high = cmax)
    ci_draw = np.random.uniform(low = cimin, high = cimax)

    #D = N12 -> D (Mortality from NYHA 1/2)
    dmin = 0.0116
    dmax = 0.0126
    # convert min-max values with RR's
    dimin = RR_intervention(dmin, RR_mortality, 12, 1)
    dimax = RR_intervention(dmax, RR_mortality, 12, 1)
    dc_draw = np.random.uniform(low = dmin, high = dmax)
    di_draw = np.random.uniform(low = dimin, high = dimax)

    #A = N12 -> N12 (residual; No change from NYHA 1/2)
    ac = 1 - bc_draw - cc_draw - dc_draw
    ai = 1 - bi_draw - ci_draw - di_draw

    #E = N34 -> N12 (No recovery from NYHA 3/4 to 
    #NYHA 1/2 was assumed 0)
    ec = 0
    ei = 0

    #G = N34 -> H (Hospital readmission rate from NYHA 3/4)
    gmin = 0.0036
    gmax = 0.1680
    gimin = RR_intervention(gmin, RR_readmission, 12, 1)
    gimax = RR_intervention(gmax, RR_readmission, 12, 1)
    gc_draw = np.random.uniform(low = gmin, high = gmax)
    gi_draw = np.random.uniform(low = gimin, high = gimax)

    #H = N34 -> D (Mortality rate from NYHA 3/4)
    hmin = 0.0215
    hmax = 0.0381
    himin = RR_intervention(hmin, RR_mortality, 12, 1)
    himax = RR_intervention(hmax, RR_mortality, 12, 1)
    hc_draw = np.random.uniform(low = hmin, high = hmax)
    hi_draw = np.random.uniform(low = himin, high = himax)
    
    #F = N34 -> N34 (resultant; No change from NYHA 3/4)
    fc = 1 - gc_draw - hc_draw
    fi = 1 - gi_draw - hi_draw

    #J = H -> N34 (Discharge to NYHA 3/4)
    jmin = 0.0350
    jmax = 0.1050
    jc_draw = np.random.uniform(low = jmin, high = jmax)
    ji_draw = jc_draw#np.random.uniform(low = jimin, high = jimax)

    #K = H -> H (Hospital admission were 
    # not longer than 1 month; defined 0)
    kc = 0
    ki = 0

    #L = H -> D (In hospital mortality)
    lmin = 0.0395
    lmax = 0.1185
    lc_draw = np.random.uniform(low = lmin, high = lmax)
    li_draw = lc_draw #np.random.uniform(low = limin, high = limax)
    
    #I = H -> N12 (resultant; discharge to NYHA 1/2)
    ic = 1 - jc_draw - kc - lc_draw
    ii = 1 - ji_draw - ki - li_draw
    #M,N,O define as zero, P defined as 1 (dead = dead)
    
    # returns proababilities for both control and intervention cohort
    return ac, bc_draw, cc_draw, dc_draw, \
    ec, fc, gc_draw, hc_draw, ic, jc_draw, kc, lc_draw,\
    ai, bi_draw, ci_draw, di_draw, \
    ei, fi, gi_draw, hi_draw, ii, ji_draw, ki, li_draw 

# function inhereted from baseline model
def infl_adjustment(months, yearly_CPI=1.029): #months
    """
    Compute a inflation adjustment factor for the amount
    of months (months) that have passed since the 
    start year (reference year; 2020). Use a predefined
    inflation factor (yearly_CPI).
    
    Output: The inflation adjustment factor.
    """
    
    CPI_adj_factor = yearly_CPI**(months/12)
    return CPI_adj_factor

# costs drawn from uniform distribution, 
# min and max values available inside function
def draw_unif_costs(ic, CPI, refyear):
    """
    - ic: Either the 'Intervention' or 'Control' arm 
        as follow-up costs can differ.
    - CPI: Consumer price index adjustment factor, 
        used to compute the current
        costs indexed from the year in which costs were computed.
        In the study either 2014 or 2016 were 
        used for different costs.
    - refyear: The refernce case year in which the simulations start,
        in the study 2020 was used.

    Output: Cost per month for each of the 4 Markov States
    """
        
    c_fu_min_c, c_fu_max_c = 13, 177
    c_fu_min_i, c_fu_max_i = 10, 125
    
    FU_cost_min = c_fu_min_c*(CPI**(refyear-2016))
    FU_cost_max = c_fu_max_c*(CPI**(refyear-2016))
    FU_cost = round(np.random.uniform(low = FU_cost_min, high = FU_cost_max),2)
    if ic=='Intervention':
        FU_cost_min_i = c_fu_min_i*(CPI**(refyear-2014))
        FU_cost_max_i = c_fu_max_i*(CPI**(refyear-2014))
        # add intervention costs on top of outpatient clinic FU costs
        FU_cost = round(np.random.uniform(low = FU_cost_min_i, high = FU_cost_max_i),2)

    Costs_N12 = FU_cost
    Costs_N34 = FU_cost
    Costs_H = round(3800*(CPI**(refyear-2016)),2)
    Costs_D = 0
    
    return Costs_N12, Costs_N34, Costs_H, Costs_D

# QALYs per month drawn from uniform distribution
def draw_unif_QALYs():
    """
    Define monthly QALYs for the 4 Markov 
    states used in this model.
    
    Output: QALYs per month for each of the 4 Markov states
     """
    Qmin_N12 = 0.059
    Qmax_N12 = 0.069
    Q_N12 = np.random.uniform(low = Qmin_N12, high = Qmax_N12)
    
    Qmin_N34 = 0.038
    Qmax_N34 = 0.059
    Q_N34 = np.random.uniform(low = Qmin_N34, high = Qmax_N34)
    
    Qmin_H = Qmin_N34
    Qmax_H = Qmax_N34
    Q_H = np.random.uniform(low = Qmin_H, high = Qmax_H)
    Q_D = 0
    
    return Q_N12, Q_N34, Q_H, Q_D



## Simulate a month
The function below simulates a single period (month) based on input transition probabilities in a matrix and then calculates the QALYs and Costs with discounting.

In [12]:
# same function as in baseline model
def simulate_month(df, # pd DataFrame where all results are stored
                   r, # A name to add to each row of new results in df
                   month, # the period (month) since start of simulation
                   patient_dist, # Markov state distribution before new period
                   transition_mat, # Matrix with transition probabilities
                   Q_mat, # Matrix with QALYs per Markov state
                   C_mat, # Matrix with Costs per Markov state
                   discount_rate_C, # Discounting % for costs
                   CPI, # Inflation rate
                   discount_rate_Q): # Discounting % for QALYs
    """
    Simulates a single month given input parameters
    
    Output: df with results (df) and new Markov state
    distribution of patients
    """
    
    # Compute inflation adjustment factor (CPI_adj)
    # for the amount of months that have passed since 
    # the begin of simulations
    CPI_adj = infl_adjustment(month, CPI)
    
    # compute the patient Markov state distribution after 1 period (month)
    new_patient_dist = np.matmul(patient_dist,transition_mat)
    
    # Use the patient Markov state distribution 
    #to compute costs and QALYs for the specified period (month)
    QALYs = new_patient_dist*Q_mat
    Costs = new_patient_dist*C_mat
    T_Q = QALYs.sum()
    T_C = Costs.sum()
    # Compute discounted costs and QALYs
    disc_factor_C = discount_rate_C**(month/12)
    disc_factor_Q = discount_rate_Q**(month/12)
    disc_Q = T_Q/(disc_factor_Q )
    disc_C = round((T_C*CPI_adj)/(disc_factor_C), 2)
    
    # put everything in a new row in the dataframe
    nr = [r, month, *new_patient_dist, 
          *QALYs, T_Q, disc_Q, 
          *Costs, T_C, disc_C]
    df.loc[len(df)]=nr
    
    return df, new_patient_dist


## Perform the simulation repeatedly
The function below implements simulation af both a control and corresponding intervention cohort for multiple periods (months) given a specified transition matrix. This simulation is repeated several times.

In [22]:
# note: Q and C are now drawn once while in previous versions those differed?
def simulate_uniform_dist(repeats):
    
    """
    Simulates results for control and 
    intervention arm based on pre-defined 
    amount of repeats.
    
    Output: df with results (df) and new Markov state
    distribution of patients
    """
    
    t1 = time.time()

    sim_months = 60

    discount_rate_C = 1.04
    discount_rate_Q = 1.015
    CPI = 1.029
    refyear = 2020

    cohort_size = 1e5
    cols = ['Repeat', 'Month', 'NYHA_12', 'NYHA_34', 'Hospital', 'Dead', 
                'Q_N12','Q_N34','Q_H', 'Q_D', 'QALY_tot', 'QALY_disc',
                'C_N12', 'C_N34','C_H', 'C_D', 'Cost_tot', 'Cost_disc']

    control_result_df = pd.DataFrame(columns = cols)
    
    intervention_result_df = pd.DataFrame(columns = cols)
    t2 = time.time()
    for r in range(0,repeats):
        # for each repeat restart the cohort distribution
        # and draw all new parameters
        cohort_dist_control = np.array([0,0,cohort_size,0])
        cohort_dist_intervention = np.array([0,0,cohort_size,0])
        
        # input probabilities for events in transition matrices
        ac, bc, cc, dc, ec, fc, gc, hc, ic, jc, kc, lc,\
        ai, bi, ci, di, ei, fi, gi, hi, ii, ji, ki, li = \
        draw_unif_probabilities()
        
        # QALY anc Cost matrices
        QALY_N12c, QALY_N34c, QALY_Hc, QALY_Dc = draw_unif_QALYs()
        Q_matc = np.array([QALY_N12c,QALY_N34c,QALY_Hc,QALY_Dc])
        Q_mati = Q_matc 
        
        C_matc = np.array(list(draw_unif_costs('Control', CPI, refyear)))
        C_mati = np.array(list(draw_unif_costs('Intervention', CPI, refyear))) 
            
        # transition matrices for control
        # and intervention arm
        trmat_control = \
        np.array([[ac, bc, cc, dc], 
                  [ec, fc, gc, hc], 
                  [ic, jc, kc, lc],
                  [0,0,0,1]], dtype = 'float64')
        
        trmat_intervention = \
        np.array([[ai, bi, ci, di], 
                  [ei, fi, gi, hi], 
                  [ii, ji, ki, li],
                  [0,0,0,1]], dtype = 'float64')

        for month in range(1,sim_months+1):

            control_result_df, cohort_dist_control = \
            simulate_month(control_result_df,r,month,
                           cohort_dist_control,trmat_control, 
                           Q_matc, C_matc,discount_rate_C, 
                           CPI, discount_rate_Q)
            
            intervention_result_df, cohort_dist_intervention = \
            simulate_month(intervention_result_df,r,month,
                           cohort_dist_intervention,trmat_intervention, 
                           Q_mati, C_mati, discount_rate_C, 
                           CPI, discount_rate_Q)
            
        t3 = time.time()
        if (r%200==0) & (r!=0):
            print('Finished repeat number:', r, 'running time', round((t3-t1),2), 'seconds')
        #if r ==20:
         #   break
        
    print('Total simulation time:', round((t3-t1),2), 'seconds')    
    return control_result_df, intervention_result_df
                

## Intervention vs Control arm
Compute the differences between the intervention and control arm in outcome measures (QALY, Costs, NMB) given a willingness to pay per QALY (WTP). Furhtermore, time per Markov state is computed. 

In [21]:
def delta_outcome(dfc, dfi, WTP):
    
    """
    Computes differences between control
    and intervention arm for each repeat 
    (row in dfc and dfi)
    
    Input:
    Control arm df (dfc)
    Intervention arm df (dfi)
    Willingness to pay per QALY (WTP)
    
    Output: df with all differences and computed
    per state outcomes 
    """
    
    reps = dfi.Repeat.unique()
    
    cols =['Repeat','control_costs', 'intervention_costs', 'dcosts', 
           'control_QALYs', 'intervention_QALYs', 'dQALYs', 'NMB',
           'survi','survc','N12i','N12c','N34i','N34c', 'Hi', 'Hc']
    df_out = pd.DataFrame(columns = cols)
    
    for r in reps:
        intervention = dfi[dfi.Repeat == r]
        control = dfc[dfc.Repeat == r]
        
        costi = intervention.Cost_disc.sum()/1e5
        costc = control.Cost_disc.sum()/1e5
        dcost = costi - costc
                
        QALYi = intervention.QALY_disc.sum()/1e5
        QALYc = control.QALY_disc.sum()/1e5
        dQALY = QALYi - QALYc
        NMB = WTP*dQALY-1*dcost
        
        survi = ((intervention.Dead*-1+1e5)).sum()/1e5
        survc = ((control.Dead*-1+1e5)).sum()/1e5
        
        N12i = intervention.NYHA_12.sum()/1e5
        N12c = control.NYHA_12.sum()/1e5
        
        N34i = intervention.NYHA_34.sum()/1e5
        N34c = control.NYHA_34.sum()/1e5
        
        Hi = intervention.Hospital.sum()/1e5
        Hc = control.Hospital.sum()/1e5
        
        df_out.loc[len(df_out)] = [r, costc, costi, dcost, QALYc, QALYi, dQALY, NMB,
                                  survi,survc,N12i,N12c,N34i,N34c, Hi, Hc]
        
    return df_out


## Implement functions

In [ ]:
dfc,dfi = simulate_uniform_dist(1000)
df_d = delta_outcome(dfc, dfi, 50000)
print(df_d.dcosts.values.max(), df_d.dcosts.values.min(), df_d.dcosts.values.mean(), df_d.dcosts.values.std())
df_d.head()